In [42]:
import random
from collections import Counter

In [43]:
def roll_die() -> int:
    return random.randint(1, 6)

def is_even(n: int) -> bool:
    return n % 2 == 0

def is_divisible_by_three(n: int) -> bool:
    return n % 3 == 0

def greater_than_seven(n: int) -> bool:
    return n > 7

def less_than_ten(n: int) -> bool:
    return n < 10

def sum_values(*n: int) -> int:
    return sum(n)

In [44]:
from enum import Enum, auto

class State(Enum):
    RED = auto()
    GREEN = auto()

### Naive, Imperative Implementation

In [45]:
def game() -> State:
    value = roll_die()

    state = State.GREEN if is_even(value) else State.RED

    if state is State.GREEN:
        value = sum_values(roll_die(), roll_die())
    else:
        value = roll_die()
        value *= 2
        if greater_than_seven(value):
            state = State.GREEN
    
    if state is State.GREEN and less_than_ten(value):
        state = State.RED

    if state is State.RED:
        value = sum_values(roll_die(), roll_die(), roll_die())
        if is_divisible_by_three(value):
            state = State.GREEN
    
    return state

In [46]:
results = [game() for _ in range(1_000)]
Counter(results)

Counter({<State.GREEN: 2>: 509, <State.RED: 1>: 491})

### Naive, Functional Implementation

In [47]:
def step_1(value: int) -> State:
    return State.GREEN if is_even(value) else State.RED

def step_2(_: int) -> int:
    return sum_values(roll_die(), roll_die())

def step_3(_: int) -> int:
    return roll_die() * 2

def step_4(value: int) -> State:
    return State.GREEN if greater_than_seven(value) else State.RED

def step_5(value: int) -> State:
    return State.RED if less_than_ten(value) else State.GREEN

def step_6(_: int) -> State:
    value = sum_values(roll_die(), roll_die(), roll_die())
    return State.GREEN if is_divisible_by_three(value) else State.RED

def game_() -> State:
    value = roll_die()

    state = step_1(value)

    if state is State.GREEN:
        value = step_2(value)
    else:
        value = step_3(value)
        state = step_4(value)
    
    if state is State.GREEN:
        state = step_5(value)

    if state is State.RED:
        state = step_6(value)
    
    return state

In [48]:
for i in range(1_000):
    random.seed(i)
    value = game()
    
    random.seed(i)
    value_ = game_()
    # print(value, value_)
    assert value is value_, i

### Pure Python Implementation

In [49]:
from dataclasses import dataclass
import functools
from typing import Any, Callable, Union, TypeVar, Generic

T = TypeVar("T", covariant=True)

@dataclass
class Success(Generic[T]):
    value: T

@dataclass
class Failure(Generic[T]):
    value: T


T1 = TypeVar("T1")
T2 = TypeVar("T2")
T3 = TypeVar("T3")

Result = Union[Success[T1], Failure[T2]]

def compose(*functions: Callable[..., Any]) -> Callable[..., Any]:
    def compose_inner(f: Callable[[T1], T2], g: Callable[[T2], T3]) -> Callable[[T1], T3]:
        return lambda x: g(f(x))
    return functools.reduce(
        compose_inner,
        functions,
        identity
    )

def invert(value: Result[T1, T2]) -> Result[T2, T1]:
    return Failure(value.value) if isinstance(value, Success) else Success(value.value)

def identity(x: T) -> T:
    return x


def bind(f: Callable[..., Result[T3, T2]]) -> Callable[[Result[T1, T2]], Result[Union[T1, T3], T2]]:
    def adapt(double_track: Result[T1, T2]) -> Result[Union[T1, T3], T2]:
        return f(double_track.value) if isinstance(double_track, Success) else  double_track
    return adapt

def map_(f: Callable[..., Success[T3]]) -> Callable[[Result[T1, T2]], Result[T3, T2]]:
    def adapt(double_track: Result[T1, T2]) -> Result[T3, T2]:
        return f(double_track.value) if isinstance(double_track, Success) else  double_track
    return adapt

def lash(f: Callable[..., Result[T1, T3]]) -> Callable[[Result[T1, T2]], Result[T1, Union[T2, T3]]]:
    def adapt(double_track: Result[T1, T2]) -> Result[T1, Union[T2, T3]]:
        return f(double_track.value) if isinstance(double_track, Failure) else  double_track
    return adapt

def alt(f: Callable[..., Failure[T3]]) -> Callable[[Result[T1, T2]], Result[T1, T3]]:
    def adapt(double_track: Result[T1, T2]) -> Result[T1, T3]:
        return f(double_track.value) if isinstance(double_track, Failure) else  double_track
    return adapt

In [50]:
def step_1(value: int) -> Result[int, int]:
    return Success(value) if is_even(value) else Failure(value)

def step_2(_: int) -> Success[int]:
    value = sum_values(roll_die(), roll_die())
    return Success(value)

def step_3(_: int) -> Failure[int]:
    value = roll_die()
    return Failure(value * 2)

def step_4(value: int) -> Result[int, int]:
    return Success(value) if greater_than_seven(value) else Failure(value)

def step_5(value: int) -> Result[int, int]:
    return Failure(value) if less_than_ten(value) else Success(value)

def step_6(_: int) -> Result[int, int]:
    value = sum_values(roll_die(), roll_die(), roll_die())
    return Success(value) if is_divisible_by_three(value) else Failure(value)

def map_result_to_state(value: Result[int, int]) -> State:
    return State.GREEN if isinstance(value, Success) else State.RED

game_2 = compose(
    step_1,
    map_(step_2),
    alt(step_3),
    lash(step_4),
    bind(step_5),
    lash(step_6),
    map_result_to_state
)
game_2(roll_die())

<State.GREEN: 2>

In [51]:
for i in range(1_000):
    random.seed(i)
    value = game()
    
    random.seed(i)
    value_ = game_2(roll_die())
    # print(value, value_)
    assert value is value_, i

### Implementing using Returns

In [52]:
%pip install returns mypy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [53]:
from returns.result import Result, Success, Failure

In [54]:
def step_1(value: int) -> Result[int, int]:
    return Success(value) if is_even(value) else Failure(value)

def step_2(_: int) -> int:
    value = sum_values(roll_die(), roll_die())
    return value

def step_3(_: int) -> int:
    value = roll_die()
    return value * 2

def step_4(value: int) -> Result[int, int]:
    return Success(value) if greater_than_seven(value) else Failure(value)

def step_5(value: int) -> Result[int, int]:
    return Failure(value) if less_than_ten(value) else Success(value)

def step_6(_: int) -> Result[int, int]:
    value = sum_values(roll_die(), roll_die(), roll_die())
    return Success(value) if is_divisible_by_three(value) else Failure(value)

def map_result_to_state(value: Result[int, int]) -> State:
    return State.GREEN if isinstance(value, Success) else State.RED

In [55]:
from returns.pipeline import pipe
from returns.pointfree import bind, lash, map_, alt

game_3 = pipe(
    step_1,
    map_(step_2),
    alt(step_3),
    lash(step_4),
    bind(step_5),
    lash(step_6),
    map_result_to_state
)
game_3(roll_die())

<State.GREEN: 2>

In [56]:
for i in range(1_000):
    random.seed(i)
    value = game()
    
    random.seed(i)
    value_ = game_3(roll_die())
    # print(value, value_)
    assert value is value_, i

### Implementing using Result

In [57]:
%pip install result

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:
from result import Ok, Err, Result

In [59]:
T1 = TypeVar("T1")
T2 = TypeVar("T2")
T3 = TypeVar("T3")

def bind(f: Callable[..., Result[T3, T2]]) -> Callable[[Result[T1, T2]], Result[Union[T1, T3], T2]]:
    def adapt(double_track: Result[T1, T2]) -> Result[Union[T1, T3], T2]:
        return f(double_track.unwrap()) if double_track.is_ok() else  double_track
    return adapt

def map_(f: Callable[..., Ok[T3]]) -> Callable[[Result[T1, T2]], Result[T3, T2]]:
    def adapt(double_track: Result[T1, T2]) -> Result[T3, T2]:
        return f(double_track.unwrap()) if isinstance(double_track, Ok) else  double_track
    return adapt

def lash(f: Callable[..., Result[T1, T3]]) -> Callable[[Result[T1, T2]], Result[T1, Union[T2, T3]]]:
    def adapt(double_track: Result[T1, T2]) -> Result[T1, Union[T2, T3]]:
        return f(double_track.unwrap_err()) if double_track.is_err() else  double_track
    return adapt

def alt(f: Callable[..., Err[T3]]) -> Callable[[Result[T1, T2]], Result[T1, T3]]:
    def adapt(double_track: Result[T1, T2]) -> Result[T1, T3]:
        return f(double_track.unwrap_err()) if isinstance(double_track, Err) else  double_track
    return adapt

In [60]:
def step_1(value: int) -> Result[int, int]:
    return Ok(value) if is_even(value) else Err(value)

def step_2(_: int) -> Ok[int]:
    value = sum_values(roll_die(), roll_die())
    return Ok(value)

def step_3(_: int) -> Err[int]:
    value = roll_die()
    return Err(value * 2)

def step_4(value: int) -> Result[int, int]:
    return Ok(value) if greater_than_seven(value) else Err(value)

def step_5(value: int) -> Result[int, int]:
    return Err(value) if less_than_ten(value) else Ok(value)

def step_6(_: int) -> Result[int, int]:
    value = sum_values(roll_die(), roll_die(), roll_die())
    return Ok(value) if is_divisible_by_three(value) else Err(value)

def map_result_to_state(value: Result[int, int]) -> State:
    return State.GREEN if value.is_ok() else State.RED

In [61]:
game_4 = compose(
    step_1,
    map_(step_2),
    alt(step_3),
    lash(step_4),
    bind(step_5),
    lash(step_6),
    map_result_to_state
)

game_4(2)

<State.GREEN: 2>

In [62]:
for i in range(1_000):
    random.seed(i)
    value = game()
    
    random.seed(i)
    value_ = game_4(roll_die())
    # print(value, value_)
    assert value is value_, i